In [1]:
import json
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

# Read the intents data from a JSON file
with open("intents.json", "r") as file:
    intents = json.load(file)

# Preprocess the data
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

def preprocess(text):
    words = nltk.word_tokenize(text.lower())
    words = [lemmatizer.lemmatize(word) for word in words if word.isalpha() and word not in stop_words]
    return " ".join(words)

# Prepare the data
corpus = []
labels = []

for intent in intents["intents"]:
    for text in intent["text"]:
        corpus.append(preprocess(text))
        labels.append(intent["intent"])

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus).toarray()
y = labels

# Split the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=0)

# Train the classifier
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

# Evaluate the classifier
score = classifier.score(X_test, y_test)
print("Accuracy: {:.2f}%".format(score * 100))

# Build the chatbot
def get_response(input_text):
    input_text = preprocess(input_text)
    input_vector = vectorizer.transform([input_text]).toarray()
    predicted_label = classifier.predict(input_vector)[0]
    for intent in intents["intents"]:
        if intent["intent"] == predicted_label:
            response = intent["responses"][0]
            return response
    return "Sorry, I don't understand"

# Test the chatbot
input_text = "hi"
response = get_response(input_text)
print(response)


Accuracy: 58.62%
Hi human, please tell me your GeniSys user


In [2]:
import json
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM

# Read the intents data from a JSON file
with open("intents.json", "r") as file:
    intents = json.load(file)

# Preprocess the data
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

def preprocess(text):
    words = nltk.word_tokenize(text.lower())
    words = [lemmatizer.lemmatize(word) for word in words if word.isalpha() and word not in stop_words]
    return " ".join(words)


corpus = []
labels = []

for intent in intents["intents"]:
    for text in intent["text"]:
        corpus.append(preprocess(text))
        labels.append(intent["intent"])

# Tokenize the data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
word_index = tokenizer.word_index
vocab_size = len(word_index)

X = tokenizer.texts_to_sequences(corpus)
X = pad_sequences(X)

y = np.zeros((len(labels), len(set(labels))))
for i, label in enumerate(labels):
    y[i, list(set(labels)).index(label)] = 1

# Split the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=0)

# Build the model
model = Sequential()
model.add(Embedding(vocab_size + 1, 64, input_length=X.shape[1]))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(set(labels)), activation="softmax"))


model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
model.fit(X_train, y_train, epochs=1000, batch_size=32)

# Evaluate the model
score = model.evaluate(X_test, y_test)
print("Accuracy: {:.2f}%".format(score[1] * 100))

# Build the chatbot
def get_response(input_text):
    input_text = preprocess(input_text)
    input_seq = tokenizer.texts_to_sequences([input_text])
    input_seq = pad_sequences(input_seq, maxlen=X.shape[1])
    predicted_label = np.argmax(model.predict(input_seq), axis=-1)[0]
    for intent in intents["intents"]:
        if list(set(labels))[predicted_label] == intent["intent"]:
            response = np.random.choice(intent["responses"])
            return response
    return "Sorry, I don't understand"

# Test the chatbot
input_text = "Hello there"
response = get_response(input_text)
print(response)


Epoch 1/1000
4/4 [==============================] - 3s 18ms/step - loss: 3.0925 - accuracy: 0.0263
Epoch 2/1000
4/4 [==============================] - 0s 19ms/step - loss: 3.0907 - accuracy: 0.0614
Epoch 3/1000
4/4 [==============================] - 0s 17ms/step - loss: 3.0887 - accuracy: 0.0526
Epoch 4/1000
4/4 [==============================] - 0s 16ms/step - loss: 3.0866 - accuracy: 0.0965
Epoch 5/1000
4/4 [==============================] - 0s 16ms/step - loss: 3.0852 - accuracy: 0.0789
Epoch 6/1000
4/4 [==============================] - 0s 17ms/step - loss: 3.0778 - accuracy: 0.0877
Epoch 7/1000
4/4 [==============================] - 0s 16ms/step - loss: 3.0776 - accuracy: 0.1404
Epoch 8/1000
4/4 [==============================] - 0s 19ms/step - loss: 3.0753 - accuracy: 0.0702
Epoch 9/1000
4/4 [==============================] - 0s 21ms/step - loss: 3.0723 - accuracy: 0.1053
Epoch 10/1000
4/4 [==============================] - 0s 15ms/step - loss: 3.0611 - accuracy: 0.0789
Epoch 11/

4/4 [==============================] - 0s 17ms/step - loss: 0.9016 - accuracy: 0.7105
Epoch 84/1000
4/4 [==============================] - 0s 14ms/step - loss: 0.8629 - accuracy: 0.7368
Epoch 85/1000
4/4 [==============================] - 0s 20ms/step - loss: 0.7758 - accuracy: 0.7632
Epoch 86/1000
4/4 [==============================] - 0s 16ms/step - loss: 0.7964 - accuracy: 0.7193
Epoch 87/1000
4/4 [==============================] - 0s 15ms/step - loss: 0.8150 - accuracy: 0.7368
Epoch 88/1000
4/4 [==============================] - 0s 23ms/step - loss: 0.8189 - accuracy: 0.6930
Epoch 89/1000
4/4 [==============================] - 0s 17ms/step - loss: 0.8680 - accuracy: 0.7193
Epoch 90/1000
4/4 [==============================] - 0s 14ms/step - loss: 0.7484 - accuracy: 0.7544
Epoch 91/1000
4/4 [==============================] - 0s 16ms/step - loss: 0.7519 - accuracy: 0.7807
Epoch 92/1000
4/4 [==============================] - 0s 15ms/step - loss: 0.7134 - accuracy: 0.7719
Epoch 93/1000


4/4 [==============================] - 0s 18ms/step - loss: 0.2684 - accuracy: 0.8860
Epoch 246/1000
4/4 [==============================] - 0s 17ms/step - loss: 0.2991 - accuracy: 0.8596
Epoch 247/1000
4/4 [==============================] - 0s 20ms/step - loss: 0.2247 - accuracy: 0.9211
Epoch 248/1000
4/4 [==============================] - 0s 15ms/step - loss: 0.3026 - accuracy: 0.8860
Epoch 249/1000
4/4 [==============================] - 0s 20ms/step - loss: 0.2353 - accuracy: 0.9298
Epoch 250/1000
4/4 [==============================] - 0s 15ms/step - loss: 0.2779 - accuracy: 0.8860
Epoch 251/1000
4/4 [==============================] - 0s 15ms/step - loss: 0.2739 - accuracy: 0.8947
Epoch 252/1000
4/4 [==============================] - 0s 21ms/step - loss: 0.2784 - accuracy: 0.8860
Epoch 253/1000
4/4 [==============================] - 0s 15ms/step - loss: 0.3181 - accuracy: 0.8684
Epoch 254/1000
4/4 [==============================] - 0s 16ms/step - loss: 0.2901 - accuracy: 0.8684
Epoch

4/4 [==============================] - 0s 16ms/step - loss: 0.1911 - accuracy: 0.9035
Epoch 408/1000
4/4 [==============================] - 0s 16ms/step - loss: 0.2442 - accuracy: 0.8947
Epoch 409/1000
4/4 [==============================] - 0s 15ms/step - loss: 0.2332 - accuracy: 0.9123
Epoch 410/1000
4/4 [==============================] - 0s 16ms/step - loss: 0.2081 - accuracy: 0.9123
Epoch 411/1000
4/4 [==============================] - 0s 17ms/step - loss: 0.2535 - accuracy: 0.8684
Epoch 412/1000
4/4 [==============================] - 0s 17ms/step - loss: 0.2389 - accuracy: 0.8947
Epoch 413/1000
4/4 [==============================] - 0s 15ms/step - loss: 0.2088 - accuracy: 0.9123
Epoch 414/1000
4/4 [==============================] - 0s 17ms/step - loss: 0.2117 - accuracy: 0.8947
Epoch 415/1000
4/4 [==============================] - 0s 15ms/step - loss: 0.2045 - accuracy: 0.8772
Epoch 416/1000
4/4 [==============================] - 0s 15ms/step - loss: 0.1783 - accuracy: 0.9298
Epoch

4/4 [==============================] - 0s 16ms/step - loss: 0.2020 - accuracy: 0.9211
Epoch 570/1000
4/4 [==============================] - 0s 15ms/step - loss: 0.1852 - accuracy: 0.9123
Epoch 571/1000
4/4 [==============================] - 0s 16ms/step - loss: 0.1860 - accuracy: 0.9035
Epoch 572/1000
4/4 [==============================] - 0s 16ms/step - loss: 0.2289 - accuracy: 0.8860
Epoch 573/1000
4/4 [==============================] - 0s 16ms/step - loss: 0.2054 - accuracy: 0.9035
Epoch 574/1000
4/4 [==============================] - 0s 20ms/step - loss: 0.1842 - accuracy: 0.8947
Epoch 575/1000
4/4 [==============================] - 0s 22ms/step - loss: 0.2136 - accuracy: 0.9035
Epoch 576/1000
4/4 [==============================] - 0s 14ms/step - loss: 0.2168 - accuracy: 0.8860
Epoch 577/1000
4/4 [==============================] - 0s 20ms/step - loss: 0.1873 - accuracy: 0.8860
Epoch 578/1000
4/4 [==============================] - 0s 21ms/step - loss: 0.2071 - accuracy: 0.8860
Epoch

4/4 [==============================] - 0s 16ms/step - loss: 0.1732 - accuracy: 0.9123
Epoch 732/1000
4/4 [==============================] - 0s 15ms/step - loss: 0.1833 - accuracy: 0.8860
Epoch 733/1000
4/4 [==============================] - 0s 17ms/step - loss: 0.1748 - accuracy: 0.9035
Epoch 734/1000
4/4 [==============================] - 0s 16ms/step - loss: 0.2188 - accuracy: 0.8947
Epoch 735/1000
4/4 [==============================] - 0s 16ms/step - loss: 0.1835 - accuracy: 0.8772
Epoch 736/1000
4/4 [==============================] - 0s 15ms/step - loss: 0.1858 - accuracy: 0.9035
Epoch 737/1000
4/4 [==============================] - 0s 16ms/step - loss: 0.1978 - accuracy: 0.8772
Epoch 738/1000
4/4 [==============================] - 0s 16ms/step - loss: 0.2018 - accuracy: 0.8947
Epoch 739/1000
4/4 [==============================] - 0s 19ms/step - loss: 0.1959 - accuracy: 0.8947
Epoch 740/1000
4/4 [==============================] - 0s 20ms/step - loss: 0.2051 - accuracy: 0.8860
Epoch

4/4 [==============================] - 0s 31ms/step - loss: 0.1776 - accuracy: 0.8947
Epoch 894/1000
4/4 [==============================] - 0s 27ms/step - loss: 0.1695 - accuracy: 0.9298
Epoch 895/1000
4/4 [==============================] - 0s 19ms/step - loss: 0.1978 - accuracy: 0.8860
Epoch 896/1000
4/4 [==============================] - 0s 39ms/step - loss: 0.1882 - accuracy: 0.9035
Epoch 897/1000
4/4 [==============================] - 0s 24ms/step - loss: 0.1971 - accuracy: 0.8772
Epoch 898/1000
4/4 [==============================] - 0s 22ms/step - loss: 0.1744 - accuracy: 0.9035
Epoch 899/1000
4/4 [==============================] - 0s 15ms/step - loss: 0.1943 - accuracy: 0.8860
Epoch 900/1000
4/4 [==============================] - 0s 15ms/step - loss: 0.1861 - accuracy: 0.9211
Epoch 901/1000
4/4 [==============================] - 0s 16ms/step - loss: 0.1677 - accuracy: 0.9123
Epoch 902/1000
4/4 [==============================] - 0s 14ms/step - loss: 0.1650 - accuracy: 0.9298
Epoch

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 5, 64)             4864      
                                                                 
 lstm (LSTM)                 (None, 5, 128)            98816     
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 22)                1430      
                                                                 
Total params: 158,678
Trainable params: 158,678
Non-trai

In [4]:
# Test the chatbot
input_text = "Hi"
response = get_response(input_text)
print(response)


1/1 [==============================] - 0s 18ms/step
Hello human, please tell me your GeniSys user


In [5]:
# Test the chatbot
input_text = "why?"
response = get_response(input_text)
print(response)


1/1 [==============================] - 0s 27ms/step
Hello, I am great, how are you? Please tell me your GeniSys user


In [1]:
import tkinter as tk
from tkinter import scrolledtext
import speech_recognition as sr
# Create the main window
root = tk.Tk()
root.geometry('400x500')
root.title('Chatbot')

# Create the chat area
chat_area = scrolledtext.ScrolledText(root, wrap=tk.WORD, width=40, height=20)
chat_area.pack(padx=10, pady=10)

# Create the input field
input_frame = tk.Frame(root)
input_frame.pack(padx=10, pady=10)

input_label = tk.Label(input_frame, text='Type your message:')
input_label.pack(side=tk.LEFT)

input_field = tk.Entry(input_frame, width=20)
input_field.pack(side=tk.LEFT)


# Build the chatbot
def get_response(input_text):
    input_text = preprocess(input_text)
    input_seq = tokenizer.texts_to_sequences([input_text])
    input_seq = pad_sequences(input_seq, maxlen=X.shape[1])
    predicted_label = np.argmax(model.predict(input_seq), axis=-1)[0]
    for intent in intents["intents"]:
        if list(set(labels))[predicted_label] == intent["intent"]:
            response = np.random.choice(intent["responses"])
            return response
    return "Sorry, I don't understand"

# Create the send button
def send_message():
    message = input_field.get()
    chat_area.insert(tk.END, "You: " + message + "\n")
    input_field.delete(0, tk.END) # Clear the input field
    response = get_response(message)
    chat_area.insert(tk.END, "Chatbot: " + response + "\n")
    history_area.insert(tk.END, "You: " + message + "\n")
    history_area.insert(tk.END, "Chatbot: " + response + "\n\n")
    
send_button = tk.Button(input_frame, text='Send', command=send_message)
send_button.pack(side=tk.LEFT, padx=10)

# Create the speech button
def recognize_speech():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        r.adjust_for_ambient_noise(source)
        audio = r.listen(source)
    try:
        message = r.recognize_google(audio)
        chat_area.insert(tk.END, "You: " + message + "\n")
        response = get_response(message)
        chat_area.insert(tk.END, "Chatbot: " + response + "\n")
        history_area.insert(tk.END, "You: " + message + "\n")
        history_area.insert(tk.END, "Chatbot: " + response + "\n\n")
    except:
        chat_area.insert(tk.END, "Could not recognize your voice\n")
    
speech_button = tk.Button(input_frame, text='Speech', command=recognize_speech)
speech_button.pack(side=tk.BOTTOM, padx=10)

def preprocess(input_text):
    # Remove leading and trailing whitespace
    input_text = input_text.strip()
    return input_text

root.mainloop()
